# Genetic clock by *Danino et al. 2010*

From:

Danino, T., Mondragón-Palomino, O., Tsimring, L. et al. A synchronized quorum of genetic clocks. Nature 463, 326–330 (2010). https://doi.org/10.1038/nature08753

This oscillator produces LuxI, which produces signal AHL. AHL upregulates LuxI, GFP and AiiA production. AiiA degrades AHL.

First, install the latest version of LOICA Consortia.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Then add gene products involved in genetic networks. 

In [ ]:
oscillator = lc.GeneticNetwork()

# TODO: determine degr rate
ahl = lc.Regulator(name='AHL', init_concentration=1, degradation_rate=0.01, diffusion_rate=2.5)
luxi = lc.Regulator(name='LuxI', init_concentration=0, degradation_rate=24)
aiia = lc.Regulator(name="AiiA", degradation_rate=15)
gfp = lc.Reporter(name="yemGFP", degradation_rate=1, color='green')

oscillator.add_regulator([ahl, aiia, luxi])
oscillator.add_reporter(gfp)

# TODO: find a, K and n
ahl_luxi = lc.Hill1(ahl, luxi, alpha=[0.001, 2500], K=0.1, n=2, name='Plux')
ahl_gfp = lc.Hill1(ahl, gfp, alpha=[0.001, 2500], K=0.1, n=2, name='Plux')
ahl_aiia = lc.Hill1(ahl, aiia, alpha=[0.001, 2500], K=0.1, n=2, name='Plux')

luxi_ahl = lc.Producer(enzyme=luxi, product=ahl, k2=0.06, name='LuxI synthase')
aiia_ahl = lc.Degrader(enzyme=aiia, substrate=ahl, Km=1, k2=0.01, name="Lactonase")

# luxi_ahl = lc.Producer(enzyme=luxi, product=ahl, k2=1.1, name='LuxI synthase')
# aiia_ahl = lc.Degrader(enzyme=aiia, substrate=ahl, Km=5.6*10**-6, k2=91, name="Lactonase")

oscillator.add_operator([ahl_aiia, ahl_gfp, ahl_luxi])
oscillator.add_impactor([luxi_ahl, aiia_ahl])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=200)
oscillator.draw(pos=nx.planar_layout)

Next, add circuit to strain with defined metabolism

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
clock_strain = lc.Strain(name="E. coli", genetic_network=oscillator, metabolism=metab)

In [ ]:
sample = lc.Sample(strain=clock_strain)
sample.set_extracel_degr(chemical_name='AHL', ext_degr_rate=0.01)
assay = lc.Assay([sample], 
                n_measurements=200, 
                interval=1/60,
                name='Quorum clock',
                description='Simulated GFP expression of quorum clock by Danino et al. (2010)'
                )

In [ ]:
print(0/1)

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular AHL'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='AHL in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='yemGFP in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='AiiA in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='LuxI in E. coli'].plot(x='Time', y='Measurement', style='-', ax=ax)
# m[m.Signal=='E. coli biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
plt.legend(['Extracellular AHL', 'Internal AHL', 'GFP', 'AiiA', 'LuxI']) #, 'Biomass'